In [4]:
import pickle
import pandas as pd
from my_greedy_heuristic import MyGreedyHeuristic  # Asegúrate de que el archivo esté en el mismo directorio o agrega el path
# my_greedy_heuristic_v1.py
from skeleton_knapsack import KnapsackSkeleton
import time

In [5]:
# 1️⃣ Cargar el DataFrame con las muestras
ruta = r"salida_muestras\lotes_100_df.pkl"

with open(ruta, "rb") as f:
    df_recuperado = pickle.load(f)

print(type(df_recuperado))
print(df_recuperado.head())

<class 'pandas.core.frame.DataFrame'>
   lote_id  num_items  capacidad  total_pesos  \
0        0        400       6502        10073   
1        1        400       4518         9903   
2        2        400       5606        10221   
3        3        400       6620        10353   
4        4        400       4734        10891   

                                               pesos  \
0  [11, 23, 3, 8, 23, 21, 39, 13, 10, 18, 15, 40,...   
1  [48, 8, 5, 41, 46, 48, 15, 47, 6, 42, 30, 46, ...   
2  [38, 40, 1, 5, 47, 4, 2, 46, 41, 36, 47, 29, 3...   
3  [19, 39, 10, 33, 37, 46, 43, 2, 26, 46, 44, 22...   
4  [29, 47, 9, 13, 39, 49, 36, 27, 31, 7, 32, 46,...   

                                             valores  
0  [1, 36, 43, 18, 26, 40, 18, 2, 37, 42, 48, 14,...  
1  [26, 12, 34, 18, 7, 34, 44, 4, 6, 13, 17, 8, 4...  
2  [27, 49, 20, 3, 26, 8, 32, 1, 12, 12, 7, 16, 3...  
3  [34, 6, 28, 35, 5, 36, 21, 21, 11, 20, 28, 45,...  
4  [6, 46, 31, 47, 19, 13, 41, 44, 31, 21, 18, 32...  


In [15]:
# 2️⃣ Cargar el mejor módulo heurístico
import importlib.util

path_best = r"best_candidate_code.py"
spec = importlib.util.spec_from_file_location("best_candidate", path_best)
best_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(best_module)

print("✅ Heurística cargada correctamente desde:", path_best)

✅ Heurística cargada correctamente desde: best_candidate_code.py


In [17]:
# Mostrar la heurística seleccionada (el código fuente)
import inspect

print("📘 Heurística seleccionada por FunSearch:\n")
print(inspect.getsource(best_module.heuristic))

📘 Heurística seleccionada por FunSearch:

def heuristic(items_state):
    start_time = time.time()
    weights = items_state["weights"]
    values = items_state["values"]
    capacity = items_state["capacity"]
    N = len(weights)

    # Handle edge cases: no items or zero capacity
    if N == 0 or capacity == 0:
        return {
            "items": [],
            "total_value": 0,
            "total_peso_usado": 0,
            "solve_time": time.time() - start_time
        }

    # Dynamic Programming table: dp[w] will store the maximum value
    # that can be obtained with a knapsack of capacity 'w'.
    # This uses O(capacity) space.
    dp = [0] * (capacity + 1)

    # Build the DP table
    for i in range(N):
        w_i = weights[i]  # current item's weight
        v_i = values[i]  # current item's value

        # Iterate capacities from max down to current item's weight.
        # This ensures that when we use dp[w - w_i], it refers to the state
        # *before* considering

In [1]:
# 3️⃣ Aplicar la heurística seleccionada por FunSearch
import pandas as pd
import time

resultados = []

for idx, row in df_recuperado.iterrows():
    # Crear instancia del problema
    skeleton = KnapsackSkeleton(
        weights=row["pesos"],
        values=row["valores"],
        capacity=row["capacidad"]
    )

    # Asignar la heurística generada automáticamente por FunSearch
    skeleton.heuristic = best_module.heuristic

    # Resolver usando la heurística seleccionada
    start = time.time()
    resultado = skeleton.solve()
    end = time.time()

    # Extraer los ítems seleccionados desde el resultado (devueltos por solve)
    selected_items = resultado.get("items", [])

    # Guardar resultados en la lista
    resultados.append({
        "lote_id": row.get("lote_id", idx),
        "num_items": len(row["pesos"]),
        "capacidad": row["capacidad"],
        "valor_total": resultado.get("total_value", 0),
        "peso_usado": resultado.get("total_peso_usado", 0),
        "tiempo_segundos": end - start,
        "items_seleccionados": ",".join(map(str, selected_items))  # para CSV legible
    })

# 4️⃣ Convertir a DataFrame y mostrar resultados
df_resultados = pd.DataFrame(resultados)
print("\n✅ Resultados obtenidos con la mejor heurística de FunSearch:\n")
print(df_resultados.head())

# (Opcional) Guardar resultados
output_file = "resultados_funsearch_heuristica.csv"
df_resultados.to_csv(output_file, index=False)
print(f"\n📁 Resultados guardados en: {output_file}")



NameError: name 'df_recuperado' is not defined